In [1]:
import pandas as pd
import geopandas
import plotly.express as px
import os
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
import imageio
import mapclassify
import matplotlib
import warnings
warnings.filterwarnings("ignore")
import matplotlib.image as mpimg
%matplotlib inline
import ast

In [2]:
path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/filtered_target_0201_matrix_aggregated'
files = os.listdir(path)
times = [i[:-4] for i in files]
files = [path+'/'+i for i in files]

In [3]:
df_list = {}
for i in files:
    date = i.split('/')[-1][:-4]
    a = pd.read_csv(i)
    a = a.rename(columns={'Unnamed: 0.1': 'id'})
    df_list[date] = a

In [4]:
store_info = pd.read_csv('/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/data/all_info_after_aggregation.csv')

In [5]:
final_data = pd.read_csv('/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/data/Target_total_visitors_2018.csv')
stores = final_data.columns[2:-1].tolist()

In [6]:
abs_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/image/'
for i in stores:
    if 'aggregated' in i:
        temp_path = abs_path+i
    else:
        temp_path = abs_path+i[3:]
    folder = os.path.exists(temp_path)
    if not folder:                   
        os.makedirs(temp_path)

In [7]:
info_df = store_info.copy()
info_df['longitude'] = info_df['geo'].apply(lambda x:ast.literal_eval(x)[1])
info_df['latitude'] = info_df['geo'].apply(lambda x:ast.literal_eval(x)[0])
geometry = [Point(xy) for xy in zip(info_df.longitude, info_df.latitude)]
info_df = info_df.drop(['longitude', 'latitude'], axis=1)
info_gdf = GeoDataFrame(info_df, crs="EPSG:4326", geometry=geometry)

In [8]:
info_dict = {}
for i,cont in info_gdf.iterrows():
    info_dict[cont['safegraph_place_id']] = cont['geometry']

In [9]:
nyc_cbg = list(df_list['2019-12'].id)
nyc_cbg = [str(i) for i in nyc_cbg]

In [10]:
shp_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/tl_2017_36_bg/tl_2017_36_bg.shp'

In [11]:
shp_df = geopandas.GeoDataFrame.from_file(shp_path,encoding = 'gb18030')
#shp_df = shp_df[shp_df['GEOID'].isin(nyc_cbg)].reset_index()

In [12]:
shp_df.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,005,017500,2,360050175002,Block Group 2,G5030,S,23694,0,+40.8296335,-073.9152037,"POLYGON ((-73.91570 40.83054, -73.91485 40.830..."
1,36,005,014100,1,360050141001,Block Group 1,G5030,S,55934,0,+40.8245444,-073.9131209,"POLYGON ((-73.91661 40.82499, -73.91592 40.825..."
2,36,005,014500,1,360050145001,Block Group 1,G5030,S,47026,0,+40.8309816,-073.9050870,"POLYGON ((-73.90584 40.83106, -73.90505 40.832..."
3,36,005,007500,2,360050075002,Block Group 2,G5030,S,109058,0,+40.8202395,-073.9080678,"POLYGON ((-73.91035 40.81995, -73.91022 40.820..."
4,36,005,041800,1,360050418001,Block Group 1,G5030,S,140793,0,+40.8989195,-073.8564428,"POLYGON ((-73.86288 40.89515, -73.86146 40.897..."


In [13]:
cbg_info = {}
cbg_id = shp_df['GEOID'].tolist()
cbg_geo = shp_df['geometry'].tolist()
for i in range(len(cbg_id)):
    cbg_info[cbg_id[i]] = cbg_geo[i]

In [14]:
for k,v in df_list.items():
    v['cbg_geometry'] = v['id'].apply(lambda x: cbg_info[str(x)])

In [ ]:
def get_image(store, time):
    p = GeoDataFrame(df_list[time],geometry = 'cbg_geometry')
    fig, ax = plt.subplots(1, 1, figsize=[15,10])
    base = p.plot(column = store,legend = True,ax=ax,
                  cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white","red"]),edgecolor='black',linewidth=0.2)
    temp = info_gdf[info_gdf['safegraph_place_id'] == store]
    temp.plot(ax=ax,marker='*', color = 'blue', markersize=30,label='Target')
    plt.suptitle('Number of Visits',size = 20)
    plt.title(store+', '+time,size = 12, pad=20, loc='center')
    for x, y, label in zip(temp.geometry.x, temp.geometry.y, temp.safegraph_place_id):
        ax.annotate('Target Store', xy=(x, y), xytext=(3, 3), textcoords="offset points")
    plt.axis('off')
    if 'aggregated' in store:
        save_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/image/'+store+"/"+time+'.png'
    else:
        save_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/image/'+store[3:]+"/"+time+'.png'
    plt.savefig(save_path, dpi=100)
    #plt.show()
    #return fig

In [ ]:
for store in stores:
    for time in times:
        image_temp = get_image(store, time)
    print(store +' finished')

## yearly

In [15]:
def get_yearly_df(year, store):
    select_files = [i for i in files if year in i]
    select_dfs = []
    for i in select_files:
        temp_df = pd.read_csv(i)
        temp_df = temp_df.rename(columns={'Unnamed: 0.1': 'id'})
        temp_df = temp_df[['id', store]]
        temp_df = temp_df.set_index('id')
        select_dfs.append(temp_df)
    final_df = sum(select_dfs)
    final_df = final_df.reset_index()
    final_df['cbg_geometry'] = final_df['id'].apply(lambda x: cbg_info[str(x)])
    final_df = GeoDataFrame(final_df,geometry = 'cbg_geometry')
    return final_df

In [ ]:
def save_yearly_df(year, store):
    df = get_yearly_df(year, store)
    if 'aggregated' in store:
        path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/data/yearly_tables/'+year+'/'+store+'_'+year+'.csv'
    else:
        path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/data/yearly_tables/'+year+'/'+store[3:]+'_'+year+'.csv'
    df.to_csv(path)

In [ ]:
years = ['2018', '2019', '2020']
for y in years:
    for u in stores:
        save_yearly_df(y,u)

In [ ]:
def get_yearly_figure(year, store):
    df_here = get_yearly_df(year, store)
    fig, ax = plt.subplots(1, 1, figsize=[15,10])
    base = df_here.plot(column = store,legend = True,ax=ax, cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white","red"]),
                        edgecolor='black',linewidth=0.2,k=6)
    temp = info_gdf[info_gdf['safegraph_place_id'] == store]
    temp.plot(ax=ax,marker='*', color = 'blue', markersize=30,label='Target')
    plt.suptitle('Number of Visits',size = 20)
    plt.title(store+', '+str(year),size = 12, pad=20, loc='center')
    for x, y, label in zip(temp.geometry.x, temp.geometry.y, temp.safegraph_place_id):
        ax.annotate('Target Store', xy=(x, y), xytext=(3, 3), textcoords="offset points")
    plt.axis('off')
    if 'aggregated' in store:
        save_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/yearly_image/'+year+'/'+store+"_"+year+'.png'
    else:
        save_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/yearly_image/'+year+'/'+store[3:]+"_"+year+'.png'
    plt.savefig(save_path,dpi=100)

In [ ]:
years = ['2018','2019','2020']
for year in years:
    for store in stores:
        get_yearly_figure(year, store)

## gif

In [ ]:
abs_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/image'
numbers = os.listdir(abs_path)
abs_directories = [abs_path+'/'+i for i in numbers if '.DS_Store' not in i]

In [ ]:
def get_gif(directory):
    image_list = []
    image_names = os.listdir(directory)
    images = [directory+'/'+ i for i in image_names if '.DS_Store' not in i]
    for i in images:
        image_list.append(imageio.imread(i))
    gif_save_path = '/Volumes/xuyilun/课题/Human Dynamics/safegraph/GOOGLEDRIVES/figures/Yearly_visits/gif/'+directory.split('/')[-1]+'.gif'
    imageio.mimsave(gif_save_path, image_list,duration=1.2)

In [ ]:
for p in abs_directories:
    get_gif(p)